In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from pathlib import Path

IMAGE_SIZE = 224
PATIENCE = 5
EPOCHS = 50

In [2]:
folder_path = Path("dataset")  # Change this to your folder path

# Count only subdirectories
CLASS_NUMBERS = sum(1 for entry in folder_path.iterdir() if entry.is_dir())

print(f"Number of classes: {CLASS_NUMBERS}")

Number of classes: 32


In [3]:
# Load and preprocess dataset
train_dataset = image_dataset_from_directory(
    "dataset",
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset="training"
)

# print("Class indices:", train_dataset.class_names)

val_dataset = image_dataset_from_directory(
    "dataset",
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset="validation"
)

Found 6827 files belonging to 32 classes.
Using 4779 files for training.
Found 6827 files belonging to 32 classes.
Using 2048 files for validation.


In [4]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

In [5]:
# Load MobileNetV2
base_model = MobileNetV2(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = True

In [6]:
# Fine-tune only the last 100 layers
fine_tune_at = len(base_model.layers) - 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [7]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
])

In [8]:
# Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9
)

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

base_model.trainable = True  # Fully unfreeze

model = models.Sequential([
    data_augmentation,  
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.1),  # reduced
    layers.Dense(128, activation='relu'),  # increased
    # removed L2 and BatchNorm for now
    layers.Dense(CLASS_NUMBERS, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # simpler optimizer
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# Early stopping on validation accuracy
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=PATIENCE,
    restore_best_weights=True,
    mode='max'
)

# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

# Save the model
model.save("model/banknote_classifier_mobilenetv2.h5")
print("Model training complete and saved as 'banknote_classifier_mobilenetv2.h5'.")

Epoch 1/50

150/150 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.3196 - loss: 2.5873 - val_accuracy: 0.6826 - val_loss: 1.0631
Epoch 2/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.9208 - loss: 0.3276 - val_accuracy: 0.8540 - val_loss: 0.5192
Epoch 3/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.9843 - loss: 0.0924 - val_accuracy: 0.8975 - val_loss: 0.3675
Epoch 4/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.9827 - loss: 0.0661 - val_accuracy: 0.9526 - val_loss: 0.1827
Epoch 5/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - accuracy: 0.9903 - loss: 0.0488 - val_accuracy: 0.9546 - val_loss: 0.1596
Epoch 6/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.9903 - loss: 0.0317 - val_accuracy: 0.9561 - val_loss: 0.1536
Epoch 7/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 210s 1s/step - accuracy: 0.9928 - loss: 0.0238 - val_accuracy: 0.9834 - val_loss: 0.0823
Epoch 8/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.9934 - loss: 0.0245 - val_acc

Model training complete and saved as 'banknote_classifier_mobilenetv2.h5'.
